导入用于处理的库：
1. 导入用于张量计算的pandas库
2. 从transformers库中导入BERT模型和BERT分词器
3. 导入用于构建神经网络的nn

In [1]:
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch.nn as nn
# pip install transformers -i https://pypi.tuna.tsinghua.edu.cn/simple
# pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cpu

c:\Users\ASUS\anaconda3\envs\cail\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


从HuggingFace上下载预训练模型bert-base-chinese，将其地址存于model_name备用

In [2]:
model_name = '../PretrainedModels/bert-base-chinese'

实例化一个分词器，命名为tokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained(model_name)

c:\Users\ASUS\anaconda3\envs\cail\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


这里的输入是一个列表，里面有两个文本，我们将input作为分词器的一个参数  

分词器的参数除了input外，还有下面这些：
1. padding：这里设置为True，填补分词结果中的空白处，使得分词结果的形状符合计算要求
2. max_length：用于分词的单个序列的最大值
3. truncation：设置为True后，如果输入序列的长度超过最大值，就会被截断
4. return_tensors：设置为pt，分词后得到的张量类型为PyTorch Tensors

input的分词结果保存在tokenize_input中，其为一个字典

In [4]:
input = ['根据权利书要求1所述的不锈钢复合桥面板焊接质量无损检测方法，其特征在于：包括：1）制定不锈钢复合桥面板焊接质量无损检测专项方案；2）采用一次直射波',
'本发明是一种浮动平衡式盘配流径向柱塞泵，特别适用于需要高可靠性、高压力、大流量、低噪声、平稳长寿命的场合。其旨在保证高可靠性、减小噪声，提高使用寿命。']

tokenize_input = tokenizer(input, padding=True,max_length=128, truncation=True,return_tensors='pt')

分词结果是一个字典，包括下面内容：
1. input_ids：一个张量tensor，和上面的input对应，每个文本是一个一维张量，由于有两个文本，因此整体是一个二维的，张量中的每一个数字都代表文本中的一个字或符号的词向量id，注意到每个一维张量有意义的部分的开头都是101，结尾都是102，是因为101是代表接下来内容需要池化的标记[CLS]，102是用于分割句子或者代表文本片段结束的标记[SEP]，0就是填充进去的[PAD]
2. token_type_ids：用于区分句子，不同的数字分属于不同的句子
3. attention_mask：在注意力机制中需要进行掩盖的有意义的内容标记为1，pad标记为0

In [5]:
tokenize_input

{'input_ids': tensor([[ 101, 3418, 2945, 3326, 1164,  741, 6206, 3724,  122, 2792, 6835, 4638,
          679, 7224, 7167, 1908, 1394, 3441, 7481, 3352, 4184, 2970, 6574, 7030,
         3187, 2938, 3466, 3844, 3175, 3791, 8024, 1071, 4294, 2519, 1762,  754,
         8038, 1259, 2886, 8038,  122, 8021, 1169, 2137,  679, 7224, 7167, 1908,
         1394, 3441, 7481, 3352, 4184, 2970, 6574, 7030, 3187, 2938, 3466, 3844,
          683, 7555, 3175, 3428, 8039,  123, 8021, 7023, 4500,  671, 3613, 4684,
         2198, 3797,  102,    0,    0],
        [ 101, 3315, 1355, 3209, 3221,  671, 4905, 3859, 1220, 2398, 6130, 2466,
         4669, 6981, 3837, 2520, 1403, 3393, 1853, 3808, 8024, 4294, 1166, 6844,
         4500,  754, 7444, 6206, 7770, 1377, 7479, 2595,  510, 7770, 1327, 1213,
          510, 1920, 3837, 7030,  510,  856, 1692, 1898,  510, 2398, 4937, 7270,
         2195, 1462, 4638, 1767, 1394,  511, 1071, 3192, 1762,  924, 6395, 7770,
         1377, 7479, 2595,  510, 1121, 2207, 1692, 1898

实例化一个BERT模型，命名为model

In [6]:
model = BertModel.from_pretrained(model_name)

c:\Users\ASUS\anaconda3\envs\cail\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# model(input_ids = tokenize_input['input_ids'],token_type_ids = tokenize_input['token_type_ids'],attention_mask = tokenize_input['attention_mask'])

In [8]:
output = model(**tokenize_input)

In [9]:
out = output.last_hidden_state

In [10]:
out.shape

torch.Size([2, 77, 768])

In [11]:
b = out[:, 0, :]

In [12]:
linear1 = nn.Linear(768,8)

In [13]:
b = linear1(b)

In [14]:
b.shape

torch.Size([2, 8])